# Data Cleaning for Science Education Articles

In [1]:
# Print out  all expressions
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" #default 'last_expr'
# Wider cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
# Imports
import pickle
import pandas as pd
import numpy as np

#Import regular expressions, for data processing
import re

import gensim
from gensim.parsing.preprocessing import STOPWORDS
from gensim.corpora import Dictionary
import nltk
from nltk.corpus import wordnet
from nltk import pos_tag
nltk.download('wordnet',quiet=True)
#nltk.download('punkt',quiet=True)   #required by word_tokenize method
nltk.download('averaged_perceptron_tagger',quiet=True) #required by pos_tag method

gensim.models.wrappers.dtmmodel


True

True

In [3]:
from helpers import path_pdf,path_pkl,printh,find_in_list

## Reading the datafile

First, we import the pickle file that holds all of the scraped PDF data. This data is then put into a Pandas dataframe. 

In [4]:
df = pd.read_pickle(path_pkl+'texts_norefs_V1.pkl') #read the pickle file, the first element is a Pandas dataframe
df=df.reset_index();
#Remove documents with issues, remove some columns #Note: maybe this is not needed anymore
#df = df[(df.failed == 0)]

### Start processing

In [5]:
field='processed2'

In [6]:
def ML_process(text):
    filt_text = text
    #remove 'cid'
    filt_text = re.sub('\W(cid:\d{0,3})\W', '', filt_text) #Symbols such as @    
    #remove some words in all-caps
    #USELESS NOW cause RawTextProcesser_* converted everything to lower case
    filt_text = re.sub(r'(?<=\W)(INTRODUCTION|CONCLUSION[S]?|BACKGROUND|ABSTRACT|ANALYSIS|EXPERIMENTAL|METHOD[S]?|METHODOLOGY|MOTIVATION[S]?|PRELIMINARY|RESULTS|APPLICATIONS|CONCLUDING|IMPLEMENTATION|EVALUATION|REMARKS|DISCUSSION[S]?|ACKNOWLEDGEMENTS|FUTURE PLANS|FUTURE WORK|FUTURE REASEARCH|SUMMARY|FIGURE[S]?|FIG|TABLE|I\.|II|III|IV|VI{0,3}|IX|X|XI{0,3})(?=\W)', 
                       '', filt_text)
    #remove newlines, tabs, etc. also remove digits (\d) and bullet points (\uf0b7)
    filt_text = re.sub('[\t\n\r\f\v\d\uf0b7]', ' ', filt_text)
    #removes all special characters that aren't numbers or letters
    filt_text = re.sub('[^A-Za-z0-9]+', ' ', filt_text)
    #split lines
    filt_text = re.sub('- ', '', filt_text)
    #to lower case
    filt_text = filt_text.lower()
    
    #tlie -> the
    filt_text = re.sub(' tlie ', ' the ', filt_text)
    #per cent -> percent
    filt_text = re.sub(' per cent ', ' percent ', filt_text)
    # )ed -> fied
    #filt_text = re.sub(re.escape(' \)ed '), 'fied ', filt_text)
    # - cation -> cation
    #filt_text = re.sub('- cation ', 'cation ', filt_text)
    return filt_text

Now, we apply text processing to the entire dataset, creating a processed version.  We then print an example of the processed version

In [7]:
df['processed2'] = df[field].map(lambda x: ML_process(x))

In [8]:
with open(path_pkl+'texts_norefs_clean.pkl','wb') as output: pickle.dump(df.drop(columns=['processed1']),output)

### Continue to 06 Tokenize_MakeBigrams.ipynb